In [1]:
import torch
import pandas as pd
import numpy as np

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
1
NVIDIA GeForce RTX 2060


In [2]:
n_steps = 5
n_features = 1

def split_sequence(sequence: pd.DataFrame, timestep: int):
    X = list()
    y = list()
    for i in range(len(sequence)):
        end_idx = i + timestep
        if end_idx > len(sequence) - 1:
            break
        seq_x, seq_y = sequence['Close'][i:end_idx].to_numpy(), sequence['Close'][end_idx]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


data = pd.read_csv('../datasets/BRK-A.csv')

X, y = split_sequence(data[:-10000], n_steps)
# X.shape (10000, 5)
# y.shape (10000, 1)

# (batch, n_steps, n_features) => tensorflow(keras) (batch size, number of channels, length of signal sequence.) => pytorch
X = X.reshape(X.shape[0], n_features, X.shape[1])
print(X.shape, y.shape)
print(X[0])
# X.shape (10000, 5, 1)


# ---printout---
# for a, b in zip(X, y):
#     print(a, b)

# print(data['Close'][:10])

(672, 1, 5) (672,)
[[290. 290. 290. 290. 290.]]


In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ConvNet(nn.Module):
    """Some Information about MyModule"""
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=64, groups=1, bias=True, kernel_size=2, stride=1)
        self.maxpool1 = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(2 * 64, 50)
        self.fc2 = nn.Linear(50, 1)

    def forward(self, x):
        x = self.maxpool1(F.relu(self.conv1(x)))
        x = torch.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

net = ConvNet()
print(net)

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters())

ConvNet(
  (conv1): Conv1d(1, 64, kernel_size=(2,), stride=(1,))
  (maxpool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=128, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=1, bias=True)
)


In [4]:
for epoch in range(1000):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, x in enumerate(X, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = torch.from_numpy(x)
        labels = torch.from_numpy(y)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 5 == 0:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 5:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,     1] loss: 43855.434
[1,     6] loss: 168764.575
[1,    11] loss: 112506.036
[1,    16] loss: 72757.587
[1,    21] loss: 44904.243


/opt/conda/lib/python3.7/site-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([672])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[1,    26] loss: 19258.256
[1,    31] loss: 8173.781
[1,    36] loss: 9434.462
[1,    41] loss: 16690.192
[1,    46] loss: 15721.764
[1,    51] loss: 9993.208
[1,    56] loss: 7437.821
[1,    61] loss: 7875.546
[1,    66] loss: 8149.143
[1,    71] loss: 8482.908
[1,    76] loss: 7797.360
[1,    81] loss: 7600.226
[1,    86] loss: 8286.739
[1,    91] loss: 8194.376
[1,    96] loss: 7528.767
[1,   101] loss: 7451.352
[1,   106] loss: 7494.597
[1,   111] loss: 7256.622
[1,   116] loss: 7378.109
[1,   121] loss: 7381.245
[1,   126] loss: 7287.145
[1,   131] loss: 7272.582
[1,   136] loss: 7376.857
[1,   141] loss: 7307.062
[1,   146] loss: 7299.565
[1,   151] loss: 8002.033
[1,   156] loss: 7666.450
[1,   161] loss: 7492.585
[1,   166] loss: 7922.796
[1,   171] loss: 7407.140
[1,   176] loss: 7841.638
[1,   181] loss: 8435.449
[1,   186] loss: 7691.287
[1,   191] loss: 9752.329
[1,   196] loss: 8312.385
[1,   201] loss: 7369.981
[1,   206] loss: 7855.205
[1,   211] loss: 7417.265
[1,   216

KeyboardInterrupt: 